In [4]:
def process_data(filename,major_money=1000000):
    """
    处理逐笔成交的数据返回分时图和主力资金净流入
    filname：传入的文件路径，包含了逐笔成交的信息
    major_money:自定义主力资金的界限
    return:主力资金净流入的分布和当天的分时图
    """
    df=pd.read_excel(filename)
    df.columns=df.columns.str.lower()
    df['time']=pd.to_datetime(df['time'],format='%H:%M:%S')
    # 添加一个固定的日期，例如 '2024-03-26'
    df['time'] = pd.to_datetime('2024-03-28 ' + df['time'].dt.strftime('%H:%M:%S'))
    #添加一个辅助列
    df['minute']=df['time'].dt.to_period('T')

    #划分主力资金和非主力资金
    df['major'] = df['amount'].apply(lambda x: '主力' if x > major_money else '非主力')
    # 计算资金的净流入
    df['net_amount'] = df.apply(lambda row: row['amount'] if row['type'] == '买盘' else -row['amount'], axis=1)
    df_main=df[df['major']=='主力']
    df_main_net_flow=df_main.groupby('minute')['net_amount'].sum()
    df_main_net_flow.index=df_main_net_flow.index.to_timestamp()
    df_main_net_flow.name='net_amount'


    # 画分时图，按分钟分组，并计算每个时间段的开盘价、收盘价、最高价、最低价、成交量和成交金额
    df_minute = df.groupby('minute').agg({'price': ['first', 'last', 'max', 'min'], 
                                          'volume': 'sum'})
    # 重命名列名
    df_minute.columns = ['open', 'close', 'high', 'low', 'volume']

    #把索引转化为时间序列
    df_minute.index = df_minute.index.to_timestamp()
    return df_main_net_flow,df_minute

In [5]:
import os
import glob
import pandas as pd
from scipy.stats import pearsonr

def calculate_correlation(folder_path):
    # 获取文件夹中所有文件的路径
    files = glob.glob(f"{folder_path}/*.xlsx")
    # 准备一个空的DataFrame来存储结果
    results_df = pd.DataFrame()

    # 遍历文件，计算相关系数
    for file in files:
#         # 跳过小于100KB的文件
#         if os.path.getsize(file) < 100 * 1024:
#             print(f"文件 {file} 小于100KB，已跳过。")
#             continue

        # 假设 process_data 是一个函数，用于处理数据并返回主力资金净流入和分钟价格
        main_net_flow, minute_price = process_data(file)
        # 使用内部连接对齐两个 Series（索引应为日期时间）
        # 去噪
        close = minute_price['close']
        close = close.ewm(span=20, adjust=False).mean()
        main_net_flow = main_net_flow.ewm(span=20, adjust=False).mean()
        combined_df = pd.concat([main_net_flow, close], axis=1, join='inner')
        combined_df.columns = ['Net Amount', 'Stock Price']

        # 计算皮尔逊相关系数
        correlation_coefficient, p_value = pearsonr(combined_df['Net Amount'], combined_df['Stock Price'])

        # 获取不带路径的文件名
        filename = os.path.basename(file)
        # 创建一个临时DataFrame来存储当前文件的结果
        temp_df = pd.DataFrame({'Filename': [filename], 
                                'Correlation Coefficient': [correlation_coefficient], 
                                'P-Value': [p_value]})
        # 使用concat合并DataFrame
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    # 设置文件名为索引
    results_df.set_index('Filename', inplace=True)
    return results_df



In [6]:
# 示例用法
folder_path = "D:\\沪深300股票\\2024-4-10"
correlation_results = calculate_correlation(folder_path)
print(correlation_results)

                Correlation Coefficient       P-Value
Filename                                             
000001.SZ.xlsx                 0.222051  1.143489e-02
000002.SZ.xlsx                -0.754884  8.548489e-28
000063.SZ.xlsx                -0.765210  2.101288e-27
000100.SZ.xlsx                 0.704222  1.053594e-16
000157.SZ.xlsx                 0.854013  2.241523e-16
...                                 ...           ...
600028.SH.xlsx                 0.708887  1.108975e-12
600030.SH.xlsx                -0.721071  6.848872e-29
600031.SH.xlsx                 0.053852  5.492403e-01
600036.SH.xlsx                 0.731301  4.898218e-26
600048.SH.xlsx                -0.753956  1.080509e-20

[100 rows x 2 columns]


In [7]:
# 假设 correlation_results 是包含相关系数和p值的DataFrame
correlation_results = correlation_results.reset_index()  # 重置索引以便操作'Filename'列

# 去掉“.xlsx”后缀，并替换“.SZ”和“.SH”
correlation_results['Filename'] = (correlation_results['Filename']
                                   .str.replace('.xlsx', '')  # 去掉“.xlsx”
                                   .str.replace('.SZ', '.XSHE')  # 替换“.SZ”为“.XSHE”
                                   .str.replace('.SH', '.XSHG'))  # 替换“.SH”为“.XSHG”

# 再次将'Filename'设置为索引
correlation_results.set_index('Filename', inplace=True)

# 显示修改后的DataFrame
print(correlation_results)


             Correlation Coefficient       P-Value
Filename                                          
000001.XSHE                 0.222051  1.143489e-02
000002.XSHE                -0.754884  8.548489e-28
000063.XSHE                -0.765210  2.101288e-27
000100.XSHE                 0.704222  1.053594e-16
000157.XSHE                 0.854013  2.241523e-16
...                              ...           ...
600028.XSHG                 0.708887  1.108975e-12
600030.XSHG                -0.721071  6.848872e-29
600031.XSHG                 0.053852  5.492403e-01
600036.XSHG                 0.731301  4.898218e-26
600048.XSHG                -0.753956  1.080509e-20

[100 rows x 2 columns]


In [31]:
from jqdatasdk import *
# 传入单个标的
df1 = get_valuation('000001.XSHE', end_date="2024-01-18", count=3, fields=['capitalization', 'market_cap'])
print(df1)

          code         day  capitalization  market_cap
0  000001.XSHE  2024-01-16    1.940592e+06   1812.5128
1  000001.XSHE  2024-01-17    1.940592e+06   1791.1662
2  000001.XSHE  2024-01-18    1.940592e+06   1785.3445


In [9]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(
    symbol="000001", 
    period="daily", 
    start_date="20240410", 
    end_date="20240410", 
    adjust="hfq"
)


In [43]:
def get_stock_info(code,time="20240410"):
    stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=code, period="daily", start_date=time, end_date=time, 
    adjust="hfq")
    df= ak.stock_individual_info_em(symbol=code)
    stock_individual_info_em_df = df.T.rename(columns=df.iloc[:,0]).drop("item")
    stock_individual_info_em_df.reset_index(drop=True, inplace=True)
    results_df = pd.concat([stock_zh_a_hist_df.iloc[:,-6:-1], stock_individual_info_em_df])
    return stock_individual_info_em_df
    
    

    

In [46]:
get_stock_info("000001")

KeyboardInterrupt: 

In [12]:
stock_zh_a_hist_df

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2024-04-10,1939.26,1923.01,1944.14,1921.38,1246391,1.288696e+09,1.17,-1.0,-19.5,0.64


In [13]:
stock_zh_a_hist_df.iloc[:,-6:-1]

,成交量,成交额,振幅,涨跌幅,涨跌额
0,1246391,1.288696e+09,1.17,-1.0,-19.5


In [32]:
import akshare as ak

stock_individual_info_em_df = ak.stock_individual_info_em(symbol="000001")


In [18]:
type(stock_individual_info_em_df)

pandas.core.frame.DataFrame

In [23]:
df=stock_individual_info_em_df
df

,item,value
0,总市值,209389857356.419983
1,流通市值,209385851590.499969
2,行业,银行
3,上市时间,19910403
4,股票代码,000001
5,股票简称,平安银行
6,总股本,19405918198.0
7,流通股,19405546950.0


In [27]:
df.iloc[:,0]

0     总市值
1    流通市值
2      行业
3    上市时间
4    股票代码
5    股票简称
6     总股本
7     流通股
Name: item, dtype: object

In [37]:
df_transposed = df.T.rename(columns=df.iloc[:,0]).drop("item")

print(df_transposed)

                       总市值                 流通市值  行业      上市时间    股票代码  股票简称  \
value  209389857356.419983  209385851590.499969  银行  19910403  000001  平安银行   

                 总股本            流通股  
value  19405918198.0  19405546950.0  


In [38]:
df_transposed.shape

(1, 8)